In [7]:
import numpy as np
import pandas as pd
import random
from copy import deepcopy
from sklearn.datasets import make_classification
from collections import Counter

# импортируем мои классы
from TreeCls import MyTreeClf
from LogReg import MyLogReg
from KNNcls import MyKNNClf

# импортируем метрики для OOB error
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, accuracy_score

In [8]:
class MyBaggingClf:
    def __init__(self, estimator=None, n_estimators=10, max_samples=1.0, random_state=42, oob_score=None):
        self.estimator = estimator
        self.n_estimators = n_estimators
        self.max_samples = max_samples
        self.random_state = random_state
        self.oob_score = oob_score  # Тип метрики для OOB оценки
        self.oob_score_ = None  # Хранение OOB метрики после обучения
        
    def __str__(self):
        params = vars(self) # Получаем все атрибуты экземпляра как словарь
        params_str = ', '.join(f"{key}={value}" for key, value in params.items())
        return f"MyTreeClf class: {params_str}"
    
    def fit(self, X, y):
        #Обучаем ансамбль моделей на бутстреп-выборках данных
        random.seed(self.random_state)  # Фиксируем сид для воспроизводимости бутстреп-выборок
        n_samples = X.shape[0]  # Количество строк в обучающей выборке
        rows_num_list = list(range(n_samples))
        rows_smpl_cnt = int(n_samples * self.max_samples)  # Количество строк для каждой бутстреп-выборки
        
        # Список для хранения OOB предсказаний и счетчик количества предсказаний для каждого образца
        oob_predictions = np.zeros(n_samples)
        oob_counts = np.zeros(n_samples)  # Считает, сколько раз каждый образец попал в OOB
        
        # Очищаем список моделей
        self.estimators = []
        
        for i in range(self.n_estimators):
            # Генерируем индексы для бутстреп-выборки
            sample_rows_idx = random.choices(rows_num_list, k=rows_smpl_cnt)
            oob_rows_idx = list(set(rows_num_list) - set(sample_rows_idx))  # Индексы OOB-выборки

            # Создаем подвыборку для текущего базового классификатора
            X_sample = X.iloc[sample_rows_idx]
            y_sample = y.iloc[sample_rows_idx]

            # Копируем базовую модель и обучаем её на бутстреп-выборке
            model = deepcopy(self.estimator)
            model.fit(X_sample, y_sample)

            # Сохраняем обученную модель
            self.estimators.append(model)
            
            # Предсказания для OOB-выборки
            if oob_rows_idx:
                oob_pred_proba = model.predict_proba(X.iloc[oob_rows_idx])
                oob_predictions[oob_rows_idx] += oob_pred_proba  # Накопление вероятностей для усреднения
                oob_counts[oob_rows_idx] += 1  # Учет количества предсказаний для каждого OOB образца

        # Усреднение вероятностей по каждому OOB экземпляру
        oob_avg_predictions = np.divide(oob_predictions, oob_counts, where=oob_counts > 0)
        oob_pred_labels = (oob_avg_predictions > 0.5).astype(int)  # Перевод в метки по порогу 0.5

        # Выбор метрики для расчета oob_score_
        if self.oob_score == 'accuracy':
            self.oob_score_ = accuracy_score(y, oob_pred_labels)
        elif self.oob_score == 'precision':
            self.oob_score_ = precision_score(y, oob_pred_labels)
        elif self.oob_score == 'recall':
            self.oob_score_ = recall_score(y, oob_pred_labels)
        elif self.oob_score == 'f1':
            self.oob_score_ = f1_score(y, oob_pred_labels)
        elif self.oob_score == 'roc_auc':
            self.oob_score_ = roc_auc_score(y, oob_avg_predictions)

        return self
    
    def predict(self, X, type='mean'):
        #Возвращает предсказанные классы для набора данных X.
        all_predictions = []
        
        # Собираем вероятности от каждой модели для всех строк X
        for estimator in self.estimators:
            all_predictions.append(estimator.predict_proba(X))
        
        # Преобразуем список вероятностей в массив (кол-во моделей x кол-во строк в X)
        all_predictions = np.array(all_predictions)
        
        if type == 'mean':
            # Усредняем вероятности по всем моделям и переводим в классы по порогу > 0.5
            avg_proba = np.mean(all_predictions, axis=0)
            return (avg_proba > 0.5).astype(int).tolist()
        
        elif type == 'vote':
            # Переводим вероятности в классы для каждой модели
            all_classes = (all_predictions > 0.5).astype(int)
            
            # Для каждого примера выбираем наиболее частый класс, если одинаково - возвращаем 1
            final_classes = []
            for i in range(X.shape[0]):
                votes = all_classes[:, i]
                most_common = Counter(votes).most_common()
                # Проверяем, если классы 0 и 1 встречаются одинаково, возвращаем 1, иначе возвращаем наиболее частый
                if len(most_common) > 1 and most_common[0][1] == most_common[1][1]:
                    final_classes.append(1)
                else:
                    final_classes.append(most_common[0][0])
            return final_classes

    def predict_proba(self, X):
        #Возвращает вероятности для класса 1 для каждой строки в X.
        all_probabilities = []
        
        # Собираем вероятности от каждой модели для всех строк X
        for estimator in self.estimators:
            all_probabilities.append(estimator.predict_proba(X))
        
        # Усредняем вероятности по всем моделям
        avg_proba = np.mean(all_probabilities, axis=0)
        return avg_proba.tolist()

In [ ]:
# тестирование метрик OOB
# Генерация случайных данных
np.random.seed(42)
X_test = pd.DataFrame(np.random.rand(100, 5))  # 100 образцов, 5 фичей
y_test = pd.Series(np.random.randint(0, 2, 100))  # 100 бинарных меток (0 или 1)

# Используем ваш класс KNN в качестве базового классификатора
knn_model = MyKNNClf(k=3, metric='euclidean', weight='uniform')

# Функция для тестирования MyBaggingClf с разными метриками OOB
def test_oob_score(metric_name):
    bagging_clf = MyBaggingClf(estimator=deepcopy(knn_model), n_estimators=10, max_samples=0.8, oob_score=metric_name, random_state=42)
    bagging_clf.fit(X_test, y_test)
    print(f"OOB score ({metric_name}):", bagging_clf.oob_score_)

# Тестирование для трех метрик: accuracy, precision, recall
for metric in ['accuracy', 'precision', 'recall']:
    test_oob_score(metric)

OOB score (accuracy): 0.54
OOB score (precision): 0.40625
OOB score (recall): 0.325


In [3]:
# Тестирование предсказания
# Создаем данные для тестирования
X, y = make_classification(n_samples=100, n_features=5, random_state=42)
X = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
y = pd.Series(y)

# Инициализируем базовую модель KNN
base_model = MyKNNClf(k=3)

# Инициализируем Bagging с базовой моделью
bagging_clf = MyBaggingClf(estimator=base_model, n_estimators=10, max_samples=0.8, random_state=42)
bagging_clf.fit(X, y)

# Тест 1: Предсказание с типом усреднения "mean"
mean_predictions = bagging_clf.predict(X, type='mean')
print("Тест 1 - Mean averaging")
print("Predicted classes with 'mean':", mean_predictions)

# Тест 2: Предсказание с типом усреднения "vote"
vote_predictions = bagging_clf.predict(X, type='vote')
print("\nТест 2 - Voting averaging")
print("Predicted classes with 'vote':", vote_predictions)

Тест 1 - Mean averaging
Predicted classes with 'mean': [0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1]

Тест 2 - Voting averaging
Predicted classes with 'vote': [0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1]


In [12]:
# Тестирование обучения с KNNClf
X, y = make_classification(n_samples=100, n_features=5, n_informative=3, n_redundant=0, random_state=42)
X_train = pd.DataFrame(X) 
y_train = pd.Series(y) 

# Инициализация и обучение ансамбля с использованием MyKNNClf в качестве базового классификатора
bagging_clf = MyBaggingClf(estimator=MyKNNClf(k=3, metric='euclidean', weight='uniform'), 
                           n_estimators=5, max_samples=0.8, random_state=42)
bagging_clf.fit(X_train, y_train)

# Подсчет средней суммы предсказаний для всех моделей
pred_sums = []
for model in bagging_clf.estimators:
    model_preds = model.predict_proba(X_train)
    pred_sums.append(np.sum(model_preds))

average_pred_sum = np.mean(pred_sums)

# Вывод средней суммы предсказаний для всех моделей
print("Средняя сумма предсказаний для всех моделей:", average_pred_sum)

Средняя сумма предсказаний для всех моделей: 46.93333333333334


In [ ]:
# Тестирование обучения с TreeClf
# Генерация синтетических данных
X, y = make_classification(n_samples=100, n_features=5, n_informative=3, n_redundant=0, random_state=42)
X_train = pd.DataFrame(X)
y_train = pd.Series(y)   

# Инициализация и обучение ансамбля с использованием MyTreeClf в качестве базового классификатора
bagging_clf = MyBaggingClf(estimator=MyTreeClf(max_depth=5, min_samples_split=2, max_leafs=20, criterion='entropy'), 
                           n_estimators=5, max_samples=0.8, random_state=42)
bagging_clf.fit(X_train, y_train)

# Подсчет среднего количества листьев для всех обученных деревьев
leaf_counts = [tree.leaf_count() for tree in bagging_clf.estimators]
average_leaf_count = np.mean(leaf_counts)

# Вывод средней информации о листьях
print("Среднее количество листьев для всех моделей:", average_leaf_count)

Среднее количество листьев для всех моделей: 9.4


In [7]:
# Тестирование обучения с LogLoss
# Создаём синтетический набор данных для бинарной классификации
X, y = make_classification(n_samples=100, n_features=5, n_informative=3, n_redundant=0, random_state=42)
X_train = pd.DataFrame(X) 
y_train = pd.Series(y)

# Инициализируем MyBaggingClf с MyLogReg в качестве базового классификатора
bagging_clf = MyBaggingClf(estimator=MyLogReg(n_iter=50, learning_rate=0.01), n_estimators=5, max_samples=0.8, random_state=42)

# Обучим модель на тренировочных данных
bagging_clf.fit(X_train, y_train)

# Выводим сумму весов для каждой модели в ансамбле
for i, model in enumerate(bagging_clf.estimators):
    weights_sum = np.sum(model.weights)
    print(f"Сумма весов модели {i + 1}: {weights_sum}")

Start loss: 0.37
Start loss: 0.37
Start loss: 0.37
Start loss: 0.37
Start loss: 0.37
Сумма весов модели 1: 5.977673983247902
Сумма весов модели 2: 5.977673983247902
Сумма весов модели 3: 5.977673983247902
Сумма весов модели 4: 5.977673983247902
Сумма весов модели 5: 5.977673983247902


In [3]:
# Тестирование класса
sample1 = MyBaggingClf()
sample2 = MyBaggingClf(n_estimators=100)
sample3 = MyBaggingClf(max_samples=0.7)

# Проверка
print(sample1)
print(sample2)
print(sample3)

MyTreeClf class: estimator=None, n_estimators=10, max_samples=1.0, random_state=42
MyTreeClf class: estimator=None, n_estimators=100, max_samples=1.0, random_state=42
MyTreeClf class: estimator=None, n_estimators=10, max_samples=0.7, random_state=42
